In [1]:
import pandas as pd
from tqdm import tqdm
import requests
import os
import json
import time
from concurrent.futures import ThreadPoolExecutor, as_completed

In [2]:


df = pd.read_csv('Data Ekspektasi Grade FOOD - Sheet1.csv')
#drop empty rows
df = df.dropna(subset=['index'])


# Convert to key-value format with splitting
# Whitelisted values
nova_whitelist = {'1', '2', '3', '4', 'UNDEFINED'}
nutriscore_whitelist = {'A', 'B', 'C', 'D', 'E', 'UNDEFINED'}

keyValueClown = {
    row['index']: {
        "Nova Expectation": row['Nova Expectation'],
        "NutriScore Expectation": row['NutriScore Expectation'],
        "Actual Nova": [
            x.strip() for x in str(row['Actual Nova']).split(',')
            if pd.notna(x) and x.strip() in nova_whitelist
        ],
        "Actual NutriScore": [
            x.strip() for x in str(row['Actual NutriScore']).split(',')
            if pd.notna(x) and x.strip().upper() in nutriscore_whitelist
        ]
    }
    for _, row in df.iterrows()
}

df.head()

,index,Nova Expectation,NutriScore Expectation,Actual Nova,Actual NutriScore,NutriScore Accuracy,Nova Accuracy
0,Rice with Fried Chicken and Ketchup,3,D,NaN,NaN,NaN,NaN
1,Basmati Fried Rice with Fried Chicken and Sambal,3,C,NaN,NaN,NaN,NaN
2,"Bakso with Meat Stuffed Tofu, Fried Wonton Dum...",3,C,NaN,NaN,NaN,NaN
3,Black Crust Pizza with Beef Ham Italian Sausag...,3,D,NaN,NaN,NaN,NaN
4,Chocolate Ice cream drizzled with Chocolate sauce,4,D,NaN,NaN,NaN,NaN


In [13]:
json.dump(keyValueClown, open('keyValueClown.json', 'w'), indent=4)


ITEM	Nova Expectation	NutriScore Expectation	Actual Nova	Actual NutriScore
0	Rice with Fried Chicken and Ketchup	Three	D	Four	D
1	Basmati Fried Rice with Fried Chicken and Sambal	Three	C	Three	C
2	Bakso with Meat Stuffed Tofu, Fried Wonton Dum...	Three	C	Four, Three, Three	D, C, C
3	Black Crust Pizza with Beef Ham Italian Sausag...	Three	D	Four	D
4	Chocolate Ice cream drizzled with Chocolate sauce	Four	D	Four	D

In [3]:
mapping_dict_case_insensitive = {
    'UNDEFINED': 'UNDEFINED',
    'FOUR' : 4,
    'THREE' : 3,
    'TWO' : 2,
    'ONE' : 1
}

In [10]:
def upload_image_and_get_response(file_path):
    url = 'http://localhost:5128/Analysis/Method3'
    with open(file_path, 'rb') as f:
        files = {'image': (file_path, f, 'image/jpeg')}
        response = requests.post(url, files=files)
        
        # Check if the response contains JSON data
        if 'application/json' in response.headers.get('content-type', ''):
            return response.json()
        else:
            response.raise_for_status()



def update_dataframe_with_response(image_name, response_data):
    # Remove the extension from the image name
    image_name = os.path.splitext(image_name)[0]
    nova_classification = response_data['novaClassification']
    if str(nova_classification).upper() in mapping_dict_case_insensitive:
        nova_classification = mapping_dict_case_insensitive[str(nova_classification).upper()]

    if image_name not in keyValueClown:
        keyValueClown[image_name] = {
            "Actual Nova": [],
            "Actual NutriScore": []
        }

    keyValueClown[image_name]['Actual Nova'].append(nova_classification)
    keyValueClown[image_name]['Actual NutriScore'].append(response_data['nutriGrade'])

    # Save the updated data to the JSON file
    json.dump(keyValueClown, open('keyValueClown.json', 'w'), indent=4)
    


def upload_images_and_update_dataframe(directory):
    image_files = [f for f in os.listdir(directory) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp'))]

    for filename in tqdm(image_files, desc="Uploading Images"):
        success = False
        attempts = 0
        response_data = None
        while not success and attempts < 3:
            try:
                response_data = upload_image_and_get_response(os.path.join(directory, filename)) 
                success = True
            except Exception as e:
                attempts += 1
                print(f"Error processing {filename}: {e}. Attempt {attempts} of 3.")
                time.sleep(1)  # Brief pause before retry
        if not success:
            print(f"Failed to process {filename}.")
            break
        update_dataframe_with_response(filename, response_data)
        print(f"Uploaded {filename} and updated DataFrame.")

    # Export the data to a new CSV file
    df = pd.DataFrame.from_dict(keyValueClown, orient='index').reset_index()
    df.to_csv('Updated.csv', index=False)
       

# Example usage
for i in range(1, 5):
    upload_images_and_update_dataframe('images')

Uploading Images:   3%|▎         | 1/37 [00:19<11:43, 19.55s/it]

Uploaded Spaghetti with sliced meatballs.jpeg and updated DataFrame.


Uploading Images:   5%|▌         | 2/37 [00:45<13:42, 23.50s/it]

Uploaded Rice with Smashed Fried Chicken and Sambal.jpg and updated DataFrame.


Uploading Images:   8%|▊         | 3/37 [01:11<13:49, 24.39s/it]

...

Uploaded Black Crust Pizza with Beef Ham Italian Sausage Pepperoni Pork Mediterranean Black Olives Green Bell.jpg and updated DataFrame.


Uploading Images:  97%|█████████▋| 36/37 [14:51<00:24, 24.84s/it]

Uploaded Basmati Fried Rice with Fried Chicken and Sambal.jpg and updated DataFrame.


Uploading Images: 100%|██████████| 37/37 [15:15<00:00, 24.75s/it]

Uploaded Bakso with Meat Stuffed Tofu, Fried Wonton Dumplings .jpg and updated DataFrame.


In [11]:
# Function to calculate accuracy for NutriScore
def calculate_accuracy_for_nutriscore(expectation, actual):
    # Convert actual to a list if it's a string with commas
    if isinstance(actual, str) and ',' in actual:
        actual = actual.split(', ')
    elif isinstance(actual, dict) or isinstance(actual, list):
        actual = actual
    else:
        actual = [actual]
    
    # Mapping NutriScore letters to numbers
    score_mapping = {'A': 1, 'B': 2, 'C': 3, 'D': 4, 'E': 5}
    
    scores = []
    
    for act in actual:
        if expectation == act:
            scores.append(1.0)
        else:
            if expectation not in score_mapping or act not in score_mapping:
                continue
            exp_num = score_mapping[expectation]
            act_num = score_mapping.get(act, 0)  # Default to 0 if not found
            absolute_difference = abs(exp_num - act_num)
            scores.append(1 - (absolute_difference / 4))
    if len(scores) == 0:
        return None
    return sum(scores) / len(scores)

for key in list(keyValueClown.keys()):
    if 'Actual NutriScore' not in keyValueClown[key]:
        continue
    if 'NutriScore Expectation' not in keyValueClown[key]:
        continue
    keyValueClown[key]['NutriScore Accuracy'] = calculate_accuracy_for_nutriscore(
        keyValueClown[key]['NutriScore Expectation'],
        keyValueClown[key]['Actual NutriScore']
    )




In [12]:
# Function to calculate accuracy
def calculate_accuracy_for_nova(expectation, actual):
    if isinstance(actual, str) and ',' in actual:
        actual = actual.split(', ')
    elif isinstance(actual, dict) or isinstance(actual, list):
        actual = actual
    else:
        actual = [actual]
    
    
    
    scores = []
    
    for act in actual:

        if act == 'nan' or act != act or not str(act).isdigit():
            return None
        if expectation == act:
            scores.append(1.0)
        else:
            act = int(act)
            expectation = int(expectation)
            absolute_difference = abs(act - expectation)
            scores.append(1 - (absolute_difference / 3))
    if len(scores) == 0:
        return None
    return sum(scores) / len(scores) 

for key in list(keyValueClown.keys()):
    if 'Actual Nova' not in keyValueClown[key]:
        continue
    if 'Nova Expectation' not in keyValueClown[key]:
        continue
    keyValueClown[key]['Nova Accuracy'] = calculate_accuracy_for_nova(
        keyValueClown[key]['Nova Expectation'],
        keyValueClown[key]['Actual Nova']
    )

In [13]:
# To dataframe
df = pd.DataFrame.from_dict(keyValueClown, orient='index').reset_index()
df.to_csv('Updated.csv', index=False)
df.head()

,index,Nova Expectation,NutriScore Expectation,Actual Nova,Actual NutriScore,NutriScore Accuracy,Nova Accuracy
0,Rice with Fried Chicken and Ketchup,3,D,"[4, 4, 4, 4, 4, 4, 4, 4]","[D, D, C, E, D, D, D, D]",0.937500,0.666667
1,Basmati Fried Rice with Fried Chicken and Sambal,3,C,"[3, 3, 2, 3, 3, 3, 2, 3]","[D, C, B, UNDEFINED, D, C, C, D]",0.857143,0.916667
2,"Bakso with Meat Stuffed Tofu, Fried Wonton Dum...",3,C,"[3, 3, 3, 3, 3, 3, 3, 3]","[D, C, D, C, D, D, C, C]",0.875000,1.000000
3,Black Crust Pizza with Beef Ham Italian Sausag...,3,D,[],[],NaN,NaN
4,Chocolate Ice cream drizzled with Chocolate sauce,4,D,"[4, 4, 4, 4, 4, 4, 4, 4]","[D, D, D, D, D, E, D, D]",0.968750,1.000000


In [14]:
# Add row total accuracy for NOVA
# filter Null values
nova_accuracy_non_null_len = len(df[df['Nova Accuracy'].notnull()])
nova_accuracy_sum = df['Nova Accuracy'].sum()
df.loc['Total', 'Nova Accuracy'] = nova_accuracy_sum / nova_accuracy_non_null_len 


# Add row total accuracy for NutriScore
# filter Null values
nutri_score_accuracy_non_null_len = len(df[df['NutriScore Accuracy'].notnull()])
nutri_score_accuracy_sum = df['NutriScore Accuracy'].sum()
df.loc['Total', 'NutriScore Accuracy'] = nutri_score_accuracy_sum / nutri_score_accuracy_non_null_len


df.head()

,index,Nova Expectation,NutriScore Expectation,Actual Nova,Actual NutriScore,NutriScore Accuracy,Nova Accuracy
0,Rice with Fried Chicken and Ketchup,3.0,D,"[4, 4, 4, 4, 4, 4, 4, 4]","[D, D, C, E, D, D, D, D]",0.937500,0.666667
1,Basmati Fried Rice with Fried Chicken and Sambal,3.0,C,"[3, 3, 2, 3, 3, 3, 2, 3]","[D, C, B, UNDEFINED, D, C, C, D]",0.857143,0.916667
2,"Bakso with Meat Stuffed Tofu, Fried Wonton Dum...",3.0,C,"[3, 3, 3, 3, 3, 3, 3, 3]","[D, C, D, C, D, D, C, C]",0.875000,1.000000
3,Black Crust Pizza with Beef Ham Italian Sausag...,3.0,D,[],[],NaN,NaN
4,Chocolate Ice cream drizzled with Chocolate sauce,4.0,D,"[4, 4, 4, 4, 4, 4, 4, 4]","[D, D, D, D, D, E, D, D]",0.968750,1.000000


In [15]:
# Export the data to a new CSV file
df.to_csv('Computed.csv', index=False)